# 4.3. Análise dos dados de periódicos da Hemeroteca Municipal de Lisboa

Nesse notebook, vamos limpar e apresentar os dados coletados na HML. Buscaremos compreender o escopo do acervo digitalizado e ao mesmo tempo apresentar suas características gerais.


```{admonition} Atenção
:class: attention
Os dados da HML são escassos, visto que seu sítio não apresenta uma política de dados abertos. Os dados apresentados e analisados aqui são fruto de uma coleta realizada pelo [scraper_hml](../../../ferramentas-metodologicas/scrapers/HML/scraper_hml.ipynb) e representam um esforço da pesquisa, mas que não são suficientes para uma análise mais profunda, apresentando dados incompletos e insuficientes para uma análise mais profunda.
```

```{admonition} Atenção
:class: attention
Atualmente está em desenvolvimento uma nova verão da HML, e sua versão beta pode ser acessada [aqui](https://hemerotecadigital.lisboa.pt/) . A nova versão apresenta uma interface mais moderna e intuitiva, com mais recursos de filtros e busca. No entanto, a nova versão ainda não foi oficialmente lançada e não contém todo o acervo digitalizado da HML. Portanto, a análise aqui apresentada se refere à versão atual da HML, que pode ser acessada [aqui](http://hemerotecadigital.cm-lisboa.pt/).
```

## Coleta de dados

Os dados foram coletados utilizando o [scraper_hml](../../../ferramentas-metodologicas/scrapers/HML/scraper_hml.ipynb), que é um scraper desenvolvido em Python utilizando a biblioteca [BeautifulSoup](https://beautiful-soup-4.readthedocs.io/en/latest/). O scraper foi desenvolvido para coletar os dados de todos os periódicos disponíveis na HML, e os dados coletados foram salvos em um arquivo CSV.

A estrutura do HTML da HML não possui uma estrutura bem definida, e com isso há uma grande variação entre as páginas dos diretórios. Isso impossibilitou o desenvolvimento de um programa para coleta de dados mais robusto. A opção mais eficiente e possível de ser desenvolvida nessa pesquisa, foi criar um scraper para coletar as informações contidas nos metadados de cada página dos periódicos. Assim, coletamos o texto presente na tag `<title>` aninhada em `<head>`. Essa escolha se justifica após uma análise geral da estrutura das páginas e de constatar que essa tag continha as informações mais relevantes para a pesquisa: sempre constando o título do periódico e comummente o local e o período de publicação. 

Outro dado possível foi a lista de arquivos em pdf em cada página. Para isso buscamos todos os links presentes em tags `<a>` que continham o padrão `pdf` no atributo `href`. O mesmo foi realizado para as fichas históricas.

Entretanto, é importante destacar, que esses dados também oscilam e carecem de padrão, o que demandou um esforço na sua limpeza e tentativa de padronização. Para maiores informações sobre a coleta e limpeza desses dados, consulte o [scraper_hml](../../../ferramentas-metodologicas/scrapers/HML/scraper_hml.ipynb).

Os dados coletados são:

- **Título**: Título do periódico
- **Autoria**: Autoria do periódico
- **Período**: Período disponível do periódico
- **Link**: Link para o periódico
- **Fichas Históricas**: Link para as fichas históricas do periódico
- **Quantidade de PDF**: Quantidade de PDFs disponíveis do periódico
- **Links dos PDFs**: Lista com links para os PDFs disponíveis do periódico

## Apresentação dos dados

Primeiramente, vamos importar as bibliotecas necessárias para a análise.

In [1]:
# importar bibliotecas
import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.offline as py
pio.renderers.default = "notebook"

Vamos ler o arquivo CSV com os dados coletados e apresentar os dados em um DataFrame.

In [3]:
# abrir arquivo csv e criar dataframe
df = pd.read_csv('../../../data/hml/hemeroteca_lisboa.csv')

In [4]:
# visualizar as primeiras linhas do dataframe
df.head()

,Título,Anos,Local,Fichas Históricas,Quantidade de PDF,Links dos PDFs
0,Academia Portuguesa,['1933'],Sem local,Sem Ficha Histórica,11,['https://hemerotecadigital.cm-lisboa.pt/Perio...
1,O Académico: publicação bi-semanal redigida po...,[],Sem local,Sem Ficha Histórica,1,['https://hemerotecadigital.cm-lisboa.pt/Perio...
2,O academico: revista quinzenal litteraria,['1878'],Sem local,https://hemerotecadigital.cm-lisboa.pt/Periodi...,6,['https://hemerotecadigital.cm-lisboa.pt/Perio...
3,O Académico: semanário ilustrado,['1902-1903'],Sem local,https://hemerotecadigital.cm-lisboa.pt/Periodi...,8,['https://hemerotecadigital.cm-lisboa.pt/Perio...
4,AccaoColonial_NComemorativo,['1934'],Sem local,Sem Ficha Histórica,1,['https://hemerotecadigital.cm-lisboa.pt/Perio...


In [5]:
# mostrar colunas do dataframe
df.columns.tolist()

['Título',
 'Anos',
 'Local',
 'Fichas Históricas',
 'Quantidade de PDF',
 'Links dos PDFs']

### Quantidade de itens no acervo

Vamos verificar a quantidade de itens no acervo da HML, contando os valores únicos da coluna `Título`.

In [6]:
# contar valores únicos da coluna 'Título' e mostrar a soma
df['Título'].nunique()

349

### Local

Mostrar itens que possuem identificação de local.

In [7]:
# contar cada valor na coluna 'Local'
df['Local'].value_counts()

Sem local                  260
Lisboa                      57
Coimbra                      9
Rio de Janeiro               6
Porto                        5
Paris                        3
Lisboa e Rio de Janeiro      1
Praia                        1
Lourenço Marques             1
Nova Goa                     1
Washington                   1
Luanda                       1
Díli                         1
Bolama                       1
Macau                        1
Madrid                       1
Name: Local, dtype: int64

### Fichas Históricas

Vamos avaliar a quantidade de itens no acervo que não possuem fichas históricas e a porcentagem de itens que possuem fichas históricas.

In [8]:
# itens sem ficha histórica
df['Fichas Históricas'].value_counts()['Sem Ficha Histórica']

147

In [9]:
# cálculo de itens com ficha histórica
com_ficha = df['Título'].nunique()-df['Fichas Históricas'].value_counts()['Sem Ficha Histórica']
com_ficha

202

In [10]:
# porcentagem de itens com ficha histórica
porcentagem = com_ficha/df['Título'].nunique()*100
porcentagem

57.879656160458445

Ou seja, entre os 349 itens no acervo, 202 possuem fichas históricas, o que representa 57,88% do acervo. Enquanto 147 itens não possuem fichas históricas, o que representa 42,12% do acervo.

### Datas

Em relação às datas, teremos menos condições de realizar avaliações mais profundas, visto que os dados coletados carecem de padronização e estão incompletos.